# CrowdIO

## Predictor using Keras

### Imports

In [1]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

from keras.models import load_model
import cv2
from sklearn.metrics import accuracy_score, mean_squared_error, log_loss

import numpy as np
import os
import pandas
import random
import tensorflow as tf

# Instantiates the device to be used as GPU/CPU based on availability
print(tf.test.gpu_device_name())

# !mkdir TrainedModels
# !wget https://drive.google.com/open?id=1hw2ctzIj0zqUmOuroEafv8jJgUlnrCGj

Using TensorFlow backend.


/device:GPU:0


## Set Parameters

In [0]:
# Parameters
dataset_path = '/content/data.csv'
model_path = '/content/trainedModel.h5'
# Parameters

## Generate Dataset

In [0]:
def getNextDate(date):
    # Hour Overflow
    date[4] += 1
    if(date[4] > 24):
        date[4] = 0
        date[0] += 1
    # Hour Overflow
    # Day Overflow
    if(date[0] > 28 and date[0] <= 30 and date[1] in [2]):
        if(date[2] % 4 == 0 and date[0] > 29):
            date[0] = 1
            date[1] += 1
        elif(not (date % 4 == 0)):
            date[0] = 0
            date[1] += 1
    elif(date[0] > 31 and date[1] in [1,3,5,7,8,10,12]):
        date[0] = 0
        date[1] += 1
    elif(date[0] > 30 and date[1] in [4,6,9,11]):
        date[0] = 0
        date[1] += 1
    # Day Overflow
    # Month Overflow
    if(date[1] > 12):
        date[1] = 0
        date[2] += 1
    # Month Overflow

    return date

def getRandom(start, end, div=1):
    return random.randint(start,end+1)/div

def getCrowdSize(meanCrowdSize, maxDeviation, festivalMaxChange, FesticalFactor):
    crowdsize = meanCrowdSize + getRandom(-1*maxDeviation, maxDeviation)
    crowdsize += int(FesticalFactor*festivalMaxChange)
    return crowdsize

def getMaleRatio(minMaleRatio, maxMaleRatio, festFactor):
    return minMaleRatio + (maxMaleRatio - minMaleRatio)*festFactor

In [0]:
# Params
no_of_rows = 1000
input_fields = ['FestivalFactor']   
output_fields = ['CrowdSize', 'MaleRatio', 'FemaleRatio']

startDate = ['1', '1', '2019', '1']  # 'Day', 'Month', 'Year', 'Hour'

meanCrowdSize = 50
maxDeviation = 25
festivalMaxChange = 50

minMaleRatio = 0.2    # For FestivalFactor = 0
maxMaleRatio= 0.5    # For FestivalFactor = 1
# Params

In [0]:
genData = {}
curDate = startDate

# Init Arrays
genData['Day'] = []
genData['Month'] = []
genData['Year'] = []
genData['Hour'] = []
for field in input_fields:
    genData[field] = []
for field in output_fields:
    genData[field] = []

# Fill Arrays
for row in range(no_of_rows):
    genData['Day'].append(curDate[0])
    genData['Month'].append(curDate[1])
    genData['Year'].append(curDate[2])
    genData['Hour'].append(curDate[4])

    festFactor = getRandom(0, 100, 100)
    genData['FestivalFactor'].append(festFactor)

    crowdsize = getCrowdSize(meanCrowdSize, maxDeviation, festivalMaxChange, festFactor)
    genData['CrowdSize'].append(crowdsize)

    maleratio = getMaleRatio(minMaleRatio, maxMaleRatio, festFactor)
    femaleratio = 1 - maleratio

    print(row, ": CS=", crowdsize, "MR=", maleratio, "FR=", femaleratio)

    curDate = getNextDate(curDate)

## Load Dataset

In [7]:
print("Started Loading Data")
# (x_train, y_train), (x_test, y_test) = mnist.load_data()
data = pandas.read_csv(dataset_path).to_numpy()
print(data.shape)

print("Finished Loading Data")

Started Loading Data
(100, 6)
Finished Loading Data


## Preprocessing

## Build Model Using Keras

In [0]:
# Model Parameters
lr = 0.0001
rho = 0.9
epsilon = None
decay = 0.0
activation = 'tanh'

loss = keras.losses.mean_squared_error
optimizer = keras.optimizers.RMSprop(lr=lr, rho=rho, epsilon=epsilon, decay=decay)
metrics = ['accuracy']

verbose=1
epochs=10
batch_size=256
# Model Parameters

In [0]:
# Check if model doesnt exist
if (not os.path.exists(model_path + ".h5")):
    print("Started building Model")
    model = Sequential()
    model.add(Dense(img_rows*img_cols, activation=activation, input_dim=img_rows*img_cols))
    model.add(Dense(256, activation=activation))
    model.add(Dense(128, activation=activation))
    model.add(Dense(256, activation=activation))
    model.add(Dense(img_rows*img_cols, activation=activation))

    model.compile(loss=loss,
                    optimizer=optimizer,
                    metrics = metrics)
    print("Finished building Model")

    print("Started Fitting Model")
    # As Encoder, Y_train is same as X_train as we measure how the reconstructed image matches with original
    model.fit(x_train,x_train,verbose=verbose,epochs=epochs,batch_size=batch_size) 
    print("Started Fitting Model")
    print("Saving Model...")
    model.save(modelPath)
    print("Saving Model Done")
    #del model

## Load Model

In [0]:
print("Loading Model...")
model = load_model(model_path)
print("Loading Model Done")

## Test Predict

In [0]:
# Threshold Parameter
threshold = 127
# Threshold Parameter

print("Started Prediction on Test Data")

""" # For user input images
test_img_path = input("Enter test image path: ")
test_imgs = cv2.imread(test_img_path)
if (test_imgs.ndim == 3):
    test_imgs = cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)
test_imgs = cv2.threshold(test_img, threshold, 255, cv2.THRESH_BINARY)[1]
test = test_img.reshape(1, img_cols*img_rows) # x_train[1].reshape(1, img_cols*img_rows)
y_test = model.predict(test)
"""

# For test input images
y_test = []
test = []
compressed_rep_test = []
index = 0
for test_img in x_test:
    test.append(test_img.reshape(1, img_cols*img_rows)) # x_train[1].reshape(1, img_cols*img_rows)

    y_test.append(model.predict(test[index]))
        
    if (index % 1000 == 0):
        print("Predicted ", index+1, " / ", x_test.shape[0])
    index += 1
#
test = np.array(test)
compressed_rep_test = np.array(compressed_rep_test)

print("Finished Prediction on Test Data")

## Postprocessing

## Display Outputs